FINAL PROJECT: BEHAVIORIAL CLONING WOOHOO

Collaborators:
- Nicole Wong
- Audrey Li
- Elaine Pham

TODOS:
- Audrey
    1. create yellow and blue colored blocks
- Nicole
    1. reference github code to build model
- Elaine
    1. create random scene generation using yellow and blue blocks

This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/clutter.html).  I recommend having both windows open, side-by-side!

# Adding boxes and iiwa to the simulation using teleop

The below code block has the following problems:
1. the pose and angles printed don't change even when the user inputs a different teleop configuration. this may be because the simulation time is too fast and the printing cannot keep up or there is something wrong with the code. 

In [2]:
import logging
import numpy as np
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder

from manipulation import running_as_notebook
from manipulation.meshcat_utils import MeshcatPoseSliders, WsgButton
from manipulation.scenarios import (AddIiwaDifferentialIK,
                                    MakeManipulationStation)

from pydrake.all import  (Box, SpatialInertia, UnitInertia, CoulombFriction)

# # Start the visualizer.
meshcat = StartMeshcat()

def AddBox(plant):
    # add boxes
    color_list=np.array([(1, 0, 0, 1.0), (0, 1, 0, 1.0), (0, 0, 1, 1.0)])
    mu = 10.0
    for i in range(1):
        box = plant.AddModelInstance("box"+str(i))
        box_body = plant.AddRigidBody("box_body"+str(i), box,
                                    SpatialInertia(
                                        mass = 0.2,
                                        p_PScm_E=np.array([0., 0., 0.]),
                                        G_SP_E = UnitInertia(1.0, 1.0, 1.0)))
        shape = Box(.1, .1, .1)
        color=color_list[np.random.randint(0, 2)]
        X_WBox = RigidTransform(RotationMatrix(), np.array([np.random.random(), np.random.random(), 0]))
        # X_WBox = RigidTransform(RotationMatrix(), np.array([1, 1, 0]))
        if plant.geometry_source_is_registered():
            plant.RegisterCollisionGeometry(box_body, X_WBox, shape, "box_body"+str(i), CoulombFriction(mu, mu))
            plant.RegisterVisualGeometry(box_body, X_WBox, shape, "box_body"+str(i), color)

    return box


def MakeTeleopAndSimulate():
    model_directives = """
    directives:
    - add_directives:
        file: package://manipulation/iiwa_and_wsg.dmd.yaml
    - add_model:
        name: foam_brick
        file: package://drake/examples/manipulation_station/models/061_foam_brick.sdf
        default_free_body_pose:
            base_link:
                translation: [0, -0.6, 0]
    - add_model:
        name: floor
        file: package://manipulation/floor.sdf
    - add_weld:
        parent: world
        child: floor::box
        X_PC:
            translation: [0, 0, 0]
    """
    builder = DiagramBuilder()

    time_step = 0.001
    station = builder.AddSystem(
        MakeManipulationStation(model_directives, time_step=time_step, 
        prefinalize_callback=AddBox))
    plant_robot = station.GetSubsystemByName("plant")
    iiwa = plant_robot.GetModelInstanceByName("iiwa")

    controller_plant = station.GetSubsystemByName(
        "iiwa_controller").get_multibody_plant_for_control()

    # Add a meshcat visualizer.
    meshcat.ResetRenderMode()
    meshcat.DeleteAddedControls()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)

    # Set up differential inverse kinematics.
    differential_ik = AddIiwaDifferentialIK(
        builder,
        controller_plant,
        frame=controller_plant.GetFrameByName("iiwa_link_7"))
    builder.Connect(differential_ik.get_output_port(),
                    station.GetInputPort("iiwa_position"))
    builder.Connect(station.GetOutputPort("iiwa_state_estimated"),
                    differential_ik.GetInputPort("robot_state"))

    # Set up teleop widgets.
    teleop = builder.AddSystem(
        MeshcatPoseSliders(
            meshcat,
            min_range=MeshcatPoseSliders.MinRange(roll=0,
                                                    pitch=-0.5,
                                                    yaw=-np.pi,
                                                    x=-0.6,
                                                    y=-0.8,
                                                    z=0.0),
            max_range=MeshcatPoseSliders.MaxRange(roll=2 * np.pi,
                                                    pitch=np.pi,
                                                    yaw=np.pi,
                                                    x=0.8,
                                                    y=0.3,
                                                    z=1.1),
            body_index=plant_robot.GetBodyByName("iiwa_link_7").index()))
    builder.Connect(teleop.get_output_port(0),
                    differential_ik.get_input_port(0))
    builder.Connect(station.GetOutputPort("body_poses"),
                    teleop.GetInputPort("body_poses"))
    wsg_teleop = builder.AddSystem(WsgButton(meshcat))
    builder.Connect(wsg_teleop.get_output_port(0),
                    station.GetInputPort("wsg_position"))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    default_context = diagram.CreateDefaultContext()
    plant_context = plant_robot.GetMyContextFromRoot(default_context)
    station_context = station.GetMyContextFromRoot(context)

    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)
        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
            # examine the output port
            print(station.GetOutputPort("iiwa_position_measured").Eval(station_context))
            gripper_ind = plant_robot.GetBodyIndices(iiwa)[-1]
            gripper = plant_robot.get_body(gripper_ind)
            pose = gripper.EvalPoseInWorld(plant_context).translation()
            angles = plant_robot.GetPositions(plant_context, iiwa)
            print('end effector ', pose)
            print('iiwa joint angles', angles)


        meshcat.DeleteButton("Stop Simulation")
    else:
        simulator.AdvanceTo(10)

MakeTeleopAndSimulate()

INFO:drake:Meshcat listening for connections at https://a569df8b-a69d-4307-8cbd-4f24b608e3b4.deepnoteproject.com/7000/
Installing NginX server for MeshCat on Deepnote...


Keyboard Controls:
roll : KeyQ / KeyE
pitch : KeyW / KeyS
yaw : KeyA / KeyD
x : KeyJ / KeyL
y : KeyI / KeyK
z : KeyO / KeyU
Press Space to open/close the gripper
Press Escape to stop the simulation
[-0.92340549  0.11064735 -0.73952856 -1.23352896  0.0972912   1.58627993
 -0.07629873]
end effector  [ 3.53636362e-04 -4.44084375e-01  7.66707814e-01]
iiwa joint angles [-1.57  0.1   0.   -1.2   0.    1.6   0.  ]
[-0.92340665  0.11064725 -0.73952719 -1.23352895  0.09729099  1.58627992
 -0.07629857]
end effector  [ 3.53636362e-04 -4.44084375e-01  7.66707814e-01]
iiwa joint angles [-1.57  0.1   0.   -1.2   0.    1.6   0.  ]
[-0.92340664  0.11064725 -0.7395272  -1.23352895  0.09729099  1.58627992
 -0.07629858]
end effector  [ 3.53636362e-04 -4.44084375e-01  7.66707814e-01]
iiwa joint angles [-1.57  0.1   0.   -1.2   0.    1.6   0.  ]
[-0.92340662  0.11064725 -0.73952722 -1.23352896  0.09729099  1.58627992
 -0.07629858]
end effector  [ 3.53636362e-04 -4.44084375e-01  7.66707814e-01]
iiwa joint a

KeyboardInterrupt: 

## Manually generating mapping of joint angle to end effector pose

In [19]:
import numpy as np
from IPython.display import HTML
from manipulation import FindResource
from manipulation.scenarios import MakeManipulationStation
from pydrake.all import StartMeshcat
from pydrake.geometry import MeshcatVisualizer
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder, GenerateHtml

meshcat = StartMeshcat()

station = MakeManipulationStation(
    filename=FindResource("models/iiwa_and_wsg.dmd.yaml"))
plant = station.GetSubsystemByName("plant")

builder = DiagramBuilder()
builder.AddSystem(station)
MeshcatVisualizer.AddToBuilder(
      builder, station.GetOutputPort("query_object"), meshcat)
diagram = builder.Build()
simulator = Simulator(diagram)
simulator.Initialize()

# initialize context
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
station_context = station.GetMyContextFromRoot(context)

# provide initial states
# q0 = np.array([-1.57, 0.1, 0, -1.2, 0, 1.6, 0])
q0 = np.array([0, 0, 0, 0, 0, 0, 0])
iiwa = plant.GetModelInstanceByName("iiwa")
# set the joint positions of the kuka arm
plant.SetPositions(plant_context, iiwa, q0)
# examine the output port
station.GetOutputPort("iiwa_position_measured").Eval(station_context)
gripper_ind = plant.GetBodyIndices(iiwa)[-1]
gripper = plant.get_body(gripper_ind)
gripper.EvalPoseInWorld(plant_context)



INFO:drake:Meshcat listening for connections at https://a569df8b-a69d-4307-8cbd-4f24b608e3b4.deepnoteproject.com/7014/


To generate data follow the steps below:

## Automating End Effector XYZ and Joint Angles Data Collection

1. Have a nominal position to reset the robot (follow up question, will the robot ever break getting back to nominal position --> ta says check russ's clutter example for how he resets grip)

undefined. Sample random (x, y, z) end effector positions. Set roll, pitch, yaw to (3.14, 0, 1.58) and do not change. Make sure they are within the ranges ( -.25 <== x <== .3 ) , ( -.4 <== y <== .60 ) , ( table height <== z <== .78) which is the area right in front of the iiwa.

undefined. Do Diff IK in order to calculate the joint angle configurations given end effector (there can be more than one join configuration for an end effector position so is this a good check? ta said that doing diff ik is good in that it diversifies and covers more planar space. will follow up on piazza how using diff ik does this. )

undefined. After every set of (x,y,z) end effector positions, reset to the nominal position to avoid the robot getting stuck in bad positions.

In [6]:
import logging
import numpy as np
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder

from manipulation import running_as_notebook
from manipulation.meshcat_utils import MeshcatPoseSliders, WsgButton
from manipulation.scenarios import (AddIiwaDifferentialIK,
                                    MakeManipulationStation)

from pydrake.all import  (Box, SpatialInertia, UnitInertia, CoulombFriction)

# # Start the visualizer.
# meshcat = StartMeshcat()

def AddBox(plant):
    # add boxes
    color_list=np.array([(1, 0, 0, 1.0), (0, 1, 0, 1.0), (0, 0, 1, 1.0)])
    mu = 10.0
    for i in range(1):
        box = plant.AddModelInstance("box"+str(i))
        box_body = plant.AddRigidBody("box_body"+str(i), box,
                                    SpatialInertia(
                                        mass = 0.2,
                                        p_PScm_E=np.array([0., 0., 0.]),
                                        G_SP_E = UnitInertia(1.0, 1.0, 1.0)))
        shape = Box(.1, .1, .1)
        color=color_list[np.random.randint(0, 2)]
        X_WBox = RigidTransform(RotationMatrix(), np.array([np.random.random(), np.random.random(), 0]))
        # X_WBox = RigidTransform(RotationMatrix(), np.array([1, 1, 0]))
        if plant.geometry_source_is_registered():
            plant.RegisterCollisionGeometry(box_body, X_WBox, shape, "box_body"+str(i), CoulombFriction(mu, mu))
            plant.RegisterVisualGeometry(box_body, X_WBox, shape, "box_body"+str(i), color)

    return box

class Planner(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        self.DeclareAbstractOutputPort(
            "X_WG", lambda: AbstractValue.Make(RigidTransform()),
            self.CalcGripperPose)
        ## is this setting the gripper to be opened or closed
        self.DeclareVectorOutputPort("wsg_position", 1, self.CalcWsgPosition)
        ## is this just a dummy function?
        self.DeclareAbstractInputPort(
            "body_poses", AbstractValue.Make([RigidTransform()]))

    def CalcGripperPose(self, output):
        min_x, max_x = -.25, .3
        min_y, max_y = -.4, .6
        min_z, max_z = 0, .78
        rand_xyz = np.array([np.random.uniform(min_x,max_x), 
                np.random.uniform(min_y,max_y), 
                np.random.uniform(min_z,max_z)])
        # desired end effector xyz position
        X_WG_des = RigidTransform(RotationMatrix(), rand_xyz)
        output.set_value(X_WG_des)
        return 

    def CalcWsgPosition(self, context, output):
        closed = np.array([0.0])
        output.SetFromVector([closed])
        return

def MakeTeleopAndSimulate():
    model_directives = """
    directives:
    - add_directives:
        file: package://manipulation/iiwa_and_wsg.dmd.yaml
    - add_model:
        name: foam_brick
        file: package://drake/examples/manipulation_station/models/061_foam_brick.sdf
        default_free_body_pose:
            base_link:
                translation: [0, -0.6, 0]
    - add_model:
        name: floor
        file: package://manipulation/floor.sdf
    - add_weld:
        parent: world
        child: floor::box
        X_PC:
            translation: [0, 0, 0]
    """
    builder = DiagramBuilder()

    time_step = 0.001
    station = builder.AddSystem(
        MakeManipulationStation(model_directives, time_step=time_step, 
        prefinalize_callback=AddBox))
    plant_robot = station.GetSubsystemByName("plant")
    iiwa = plant_robot.GetModelInstanceByName("iiwa")

    controller_plant = station.GetSubsystemByName(
        "iiwa_controller").get_multibody_plant_for_control()

    # Add a meshcat visualizer.
    meshcat.ResetRenderMode()
    meshcat.DeleteAddedControls()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)

    # Set up differential inverse kinematics.
    differential_ik = AddIiwaDifferentialIK(
        builder,
        controller_plant,
        frame=controller_plant.GetFrameByName("iiwa_link_7"))
    builder.Connect(differential_ik.get_output_port(),
                    station.GetInputPort("iiwa_position"))
    builder.Connect(station.GetOutputPort("iiwa_state_estimated"),
                    differential_ik.GetInputPort("robot_state"))

    # Connect the sampled end effector positions --> differential ik
    planner = builder.AddSystem(Planner(plant))
    builder.Connect(planner.GetOutputPort("X_WG"),
                    differential_ik.get_input_port(0))
    # TODO: make planner.getinputport
    builder.Connect(station.GetOutputPort("body_poses"),
                    planner.GetInputPort("body_poses"))
    # TODO: make planner.getoutputport
    builder.Connect(planner.GetOutputPort("wsg_position"),
                    station.GetInputPort("wsg_position"))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    default_context = diagram.CreateDefaultContext()
    plant_context = plant_robot.GetMyContextFromRoot(default_context)
    station_context = station.GetMyContextFromRoot(context)


    old_pose = np.zeros(3)
    old_angles = np.zeros(7)
    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)
        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
            # sample random end effector positions

        meshcat.DeleteButton("Stop Simulation")
    else:
        simulator.AdvanceTo(1)

MakeTeleopAndSimulate()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a569df8b-a69d-4307-8cbd-4f24b608e3b4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>